# OpenMC models
This notebook contains the code to create and run the OpenMC model of 
a PWR pincell, based on the [example online](https://github.com/openmc-dev/openmc/wiki/Example-Jupyter-Notebooks).
Running the model includes developing one-group cross section data and depletion using 
only UOX or MOX fuel.

In [1]:
import numpy as np
import math
import openmc
import openmc.deplete as od
from openmcyclus.depletion import Depletion

In [2]:
uox_model = openmc.Model()
mox_model = openmc.Model()
openmc.config['cross_sections'] = "/home/abachmann@anl.gov/openmc-xs-data/endfb-viii.0-hdf5/cross_sections.xml"

In [3]:
# Materials
uo2 = openmc.Material(1, "uo2")
uo2.add_nuclide('U235', 0.03)
uo2.add_nuclide('U238', 0.97)
uo2.add_nuclide('O16', 2.0)
uo2.set_density('g/cm3', 10.0)
uo2.volume = 0.4778 # area of circle for fuel

zirconium = openmc.Material(name="zirconium")
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.6)

water = openmc.Material(name="h2o")
water.add_nuclide('H1', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3', 1.0)
water.add_s_alpha_beta('c_H_in_H2O')

puo2 = openmc.Material(4)
puo2.add_nuclide('Pu239', 0.94)
puo2.add_nuclide('Pu240', 0.06)
puo2.add_nuclide('O16', 2.0)
puo2.set_density('g/cm3', 11.5)

# Create the mixture
mox = openmc.Material.mix_materials([uo2, puo2], [0.97, 0.03], 'wo')
mox.volume = 0.4778 # area of circle for fuel

uox_model.Materials = openmc.Materials([uo2, zirconium, water])
mox_model.Materials = openmc.Materials([mox, zirconium, water])

In [7]:
# Geometry
fuel_outer_radius = openmc.ZCylinder(r=0.39)
clad_inner_radius = openmc.ZCylinder(r=0.40)
clad_outer_radius = openmc.ZCylinder(r=0.46)

fuel_region = -fuel_outer_radius
gap_region = +fuel_outer_radius & -clad_inner_radius
clad_region = +clad_inner_radius & -clad_outer_radius

uox_fuel = openmc.Cell(name='fuel')
uox_fuel.fill = uo2
uox_fuel.region = fuel_region

mox_fuel = openmc.Cell(name='fuel')
mox_fuel.fill = mox
mox_fuel.region = fuel_region

gap = openmc.Cell(name='air gap')
gap.region = gap_region

clad = openmc.Cell(name='clad')
clad.fill = zirconium
clad.region = clad_region

pitch = 1.26
box = openmc.model.RectangularPrism(width=pitch, height=pitch,
                               boundary_type='reflective')
water_region = -box & +clad_outer_radius
moderator = openmc.Cell(name='moderator')
moderator.fill = water
moderator.region = water_region

uox_universe = openmc.Universe(cells=(uox_fuel, gap, clad, moderator))
mox_universe = openmc.Universe(cells=(mox_fuel, gap, clad, moderator))

uox_model.geometry = openmc.Geometry(uox_universe)
mox_model.geometry = openmc.Geometry(mox_universe)

In [8]:
# Settings
settings = openmc.Settings()
settings.batches = 100
settings.inactive = 10
settings.particles = 1000
settings.output = {'tallies':True}
uox_model.settings = settings
mox_model.settings = settings

In [106]:
# Generate one-group cross section data
uox_cross_sections = od.get_microxs_and_flux(uox_model, 
                                       domains = [uo2], 
                                       chain_file="chain_endfb71_pwr.xml")
clad_cross_sections = od.get_microxs_and_flux(uox_model, 
                                       domains = [clad], 
                                       chain_file="chain_endfb71_pwr.xml")
water_cross_sections = od.get_microxs_and_flux(uox_model, 
                                       domains = [water], 
                                       chain_file="chain_endfb71_pwr.xml")
mox_cross_sections = od.get_microxs_and_flux(mox_model, 
                                       domains = [mox_fuel], 
                                       chain_file="chain_endfb71_pwr.xml")

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [75]:
# save cross sections from UOX model for use in Deplete Reactor
uox_cross_sections[1][0].to_csv("micro_xs.csv")
mox_cross_sections[1][0].to_csv("mox_xs.csv")

In [76]:
uox_microxs = od.MicroXS.from_csv("micro_xs.csv")
mox_microxs = od.MicroXS.from_csv("mox_xs.csv")

In [140]:
uox_ind_op = od.IndependentOperator(openmc.Materials([uo2]), 
                                [uox_cross_sections[0][0]],#, clad_cross_sections[0][0], water_cross_sections[0][0]],
                                [uox_cross_sections[1][0]],#, clad_cross_sections[1][0], water_cross_sections[1][0]],
                                "chain_endfb71_pwr.xml")
mox_ind_op = od.IndependentOperator(openmc.Materials([mox]), 
                                [mox_cross_sections[0][0]],
                                mox_cross_sections[1],
                                "chain_endfb71_pwr.xml")

In [141]:
uox_integrator = od.PredictorIntegrator(uox_ind_op,
                                    np.ones(12*3)*30, # 3 cycles of operation
                                   power_density = 46.3, # linear power density in W/g
                                   timestep_units='d')
uox_integrator.integrate()

[openmc.deplete] t=0.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=2592000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=5184000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=7776000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=10368000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=12960000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=15552000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=18144000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=20736000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=23328000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=25920000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=28512000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=31104000.0 s, dt=2592000.0 s, source=195.00608586449124
[openmc.deplete] t=33696000.0 s, dt

In [142]:
! mv depletion_results.h5 uox_depletion_results.h5

In [92]:
mox_integrator = od.PredictorIntegrator(mox_ind_op,
                                    np.ones(12*3)*30,
                                   power_density = 46.3, # linear power density in W/g
                                   timestep_units='d')
mox_integrator.integrate()

[openmc.deplete] t=0.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=2592000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=5184000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=7776000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=10368000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=12960000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=15552000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=18144000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=20736000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=23328000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=25920000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=28512000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=31104000.0 s, dt=2592000.0 s, source=195.77550491636865
[openmc.deplete] t=33696000.0 s, dt

In [93]:
! mv depletion_results.h5 mox_depletion_results.h5

In [143]:
# Print depletion material compositions
uox_results = od.Results("uox_depletion_results.h5")
mox_results = od.Results("mox_depletion_results.h5")

In [144]:
uox_spent = uox_results.export_to_materials(-1)
mox_spent = mox_results.export_to_materials(-1)

/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.10/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.10/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/abachmann@anl.gov/anaconda3/envs/openmcyclus-env/lib/python3.10/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)


In [145]:
#printing in format for Cyclus recipe
total = 0
weight_frac = 0
pu_frac = 0
for nuclide in uox_spent[0].nuclides:
    total += nuclide.percent
    if nuclide.percent <= 1e-10:
        continue
    Z, A, m = openmc.data.zam(nuclide.name)
    zaid = Z*1e7+A*1e4+m
    print(f"<nuclide>  <id>{int(zaid)}</id> <comp>{nuclide.percent}</comp>  </nuclide>")
    weight_frac += nuclide.percent*A
    if Z == 94:
        pu_frac += nuclide.percent*A
    total += nuclide.percent

<nuclide>  <id>80160000</id> <comp>0.04461664459672212</comp>  </nuclide>
<nuclide>  <id>922350000</id> <comp>0.0006692496689508316</comp>  </nuclide>
<nuclide>  <id>922380000</id> <comp>0.02163907262941021</comp>  </nuclide>


In [97]:
#printing in format for Cyclus recipe
for nuclide in mox_spent[0].nuclides:
    if nuclide.percent <= 1e-10:
        continue
    Z, A, m = openmc.data.zam(nuclide.name)
    zaid = Z*1e7+A*1e4+m
    print(f"<nuclide>  <id>{int(zaid)}</id> <comp>{nuclide.percent}</comp>  </nuclide>")

<nuclide>  <id>922380000</id> <comp>85.53765972559535</comp>  </nuclide>
<nuclide>  <id>922350000</id> <comp>2.612078957002141</comp>  </nuclide>
<nuclide>  <id>80160000</id> <comp>11.850261317402508</comp>  </nuclide>
